In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# Load the Excel files
fatality_file = 'bitre_fatalities_dec2024.xlsx'
crash_file = 'bitre_fatal_crashes_dec2024.xlsx'

# Read specific sheets
df_fatality = pd.read_excel(fatality_file, sheet_name='BITRE_Fatality', skiprows=4)
df_crash = pd.read_excel(crash_file, sheet_name='BITRE_Fatal_Crash', skiprows=4)

In [3]:
print("Fatality columns:", df_fatality.columns.tolist())
print("Crash columns:", df_crash.columns.tolist())

Fatality columns: ['Crash ID', 'State', 'Month', 'Year', 'Dayweek', 'Time', 'Crash Type', 'Bus Involvement', 'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement', 'Speed Limit', 'Road User', 'Gender', 'Age', 'National Remoteness Areas', 'SA4 Name 2021', 'National LGA Name 2021', 'National Road Type', 'Christmas Period', 'Easter Period', 'Age Group', 'Day of week', 'Time of day']
Crash columns: ['Crash ID', 'State', 'Month', 'Year', 'Dayweek', 'Time', 'Crash Type', 'Number Fatalities', 'Bus \nInvolvement', 'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement', 'Speed Limit', 'National Remoteness Areas', 'SA4 Name 2021', 'National LGA Name 2021', 'National Road Type', 'Christmas Period', 'Easter Period', 'Day of week', 'Time of Day']


In [4]:
# Clean column names (remove extra spaces if any)
df_fatality.columns = df_fatality.columns.str.strip()
df_crash.columns = df_crash.columns.str.strip()

# Perform LEFT JOIN on 'Crash ID'
merged_df = pd.merge(
    df_fatality,
    df_crash,
    on='Crash ID',
    how='left',
    suffixes=('_fatality', '_crash')
)

# Drop all columns that end with '_crash'
merged_df = merged_df.loc[:, ~merged_df.columns.str.endswith('_crash')]
# Rename columns by removing '_fatality' suffix
merged_df.columns = merged_df.columns.str.replace('_fatality$', '', regex=True)
# Remove truly duplicated columns (identical values)
merged_df = merged_df.loc[:, ~merged_df.T.duplicated()]


In [5]:
# Column Groups 
involvement_cols = ['Bus Involvement','Heavy Rigid Truck Involvement','Articulated Truck Involvement']
categorical_cols = ['National Remoteness Areas', 'SA4 Name 2021', 'National LGA Name 2021','National Road Type']
speed_col = 'Speed Limit'

# Clean Speed Limit 
merged_df[speed_col] = merged_df[speed_col].replace('<40', 40)
# First convert -9 to NaN
merged_df[speed_col] = merged_df[speed_col].replace(-9, np.nan)
# Then ensure the column is of nullable integer type
merged_df[speed_col] = pd.to_numeric(merged_df[speed_col], errors='coerce').astype('Int64')

# Clean Involvement Flags
for col in involvement_cols:
    merged_df[col] = merged_df[col].astype(str).apply(
        lambda x: 'Unknown' if str(x).strip().lower() in ['', 'unknown', '-9', 'nan'] else x
    )

# Clean All Categorical Columns
for col in categorical_cols:
    merged_df[col] = merged_df[col].astype(str).apply(
        lambda x: 'Unknown' if str(x).strip().lower() in ['', 'unknown', '-9', 'nan'] else x
    )

# Special: Crash Type Rule
def fix_single_crash_vehicle_flags(row):
    if str(row['Crash Type']).strip().lower() == 'single':
        yes_count = sum([row[col] == 'Yes' for col in involvement_cols])
        if yes_count > 1:
            for col in involvement_cols:
                row[col] = 'Unknown'
    return row

merged_df = merged_df.apply(fix_single_crash_vehicle_flags, axis=1)

# Define the logic for heavy vehicles involvement
# Define function for exact mapping
def get_involvement_category(row):
    bus = row['Bus Involvement']
    rigid = row['Heavy Rigid Truck Involvement']
    artic = row['Articulated Truck Involvement']
    
    # Check for fully unknown
    if bus == 'Unknown' and rigid == 'Unknown' and artic == 'Unknown':
        return 'Unknown'
    
    # Check all three are involved
    if bus == 'Yes' and rigid == 'Yes' and artic == 'Yes':
        return 'Bus, Articulated Truck and Heavy Rigid Truck Involved'
    
    # Pairwise checks
    if bus == 'Yes' and artic == 'Yes' and rigid != 'Yes':
        return 'Bus and Articulated Truck Involved as Known'
    if bus == 'Yes' and rigid == 'Yes' and artic != 'Yes':
        return 'Bus and Heavy Rigid Truck Involved as Known'
    if artic == 'Yes' and rigid == 'Yes' and bus != 'Yes':
        return 'Articulated Truck and Heavy Rigid Truck Involved as Known'
    
    # Single vehicle checks
    if bus == 'Yes' and rigid != 'Yes' and artic != 'Yes':
        return 'Only Bus Involved as Known'
    if artic == 'Yes' and bus != 'Yes' and rigid != 'Yes':
        return 'Only Articulated Truck Involved as Known'
    if rigid == 'Yes' and bus != 'Yes' and artic != 'Yes':
        return 'Only Heavy Rigid Truck Involved as Known'
    
    # All no
    if bus == 'No' and rigid == 'No' and artic == 'No':
        return 'No Heavy Vehicles Involved'

    # Default
    return 'Unknown'
    
merged_df['Vehicle Involvement'] = merged_df[
    ['Bus Involvement', 'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement']
].apply(get_involvement_category, axis=1)


# Additional Cleaning Rules

# Make the State column all caps
merged_df['State'] = merged_df['State'].str.upper()

# Age: -9 and 0 → NaN
merged_df['Age'] = pd.to_numeric(merged_df['Age'], errors='coerce')
merged_df['Age'] = merged_df['Age'].replace({-9: np.nan, 0: np.nan})

# Gender: -9 → 'Unknown'
merged_df['Gender'] = merged_df['Gender'].replace(-9, 'Unknown')
merged_df['Gender'] = merged_df['Gender'].apply(lambda x: 'Unknown' if str(x).strip().lower() == 'unknown' else x)

# Time: blank → NaN
merged_df['Time'] = merged_df['Time'].replace('', np.nan)

# Road User: 'Other' and -9 → 'Others'
merged_df['Road User'] = merged_df['Road User'].apply(
    lambda x: 'Others' if str(x).strip().lower() == 'other/-9' else x
)

# Age Group: '0_to_16' → '1_to_16'; -9 → 'Unknown'
merged_df['Age Group'] = merged_df['Age Group'].replace({'0_to_16': '1_to_16', -9: 'Unknown'})
merged_df['Age Group'] = merged_df['Age Group'].apply(
    lambda x: 'Unknown' if str(x).strip().lower() in ['-9', 'unknown', 'nan'] else x
)

# Derive 'Day of week' from 'Dayweek'
merged_df['Dayweek'] = merged_df['Dayweek'].astype(str).str.strip().str.capitalize()
merged_df['Day of week'] = merged_df['Dayweek'].apply(
    lambda x: 'Weekend' if x in ['Saturday', 'Sunday'] else 'Weekday'
)

# Specifically convert 'Undetermined' to 'Unknown' in National Road Type
merged_df['National Road Type'] = merged_df['National Road Type'].apply(
    lambda x: 'Unknown' if str(x).strip().lower() == 'undetermined' else x
)

# Fix SA4 Name when LGA is Clarence and SA4 is Unknown
merged_df.loc[
    (merged_df['National LGA Name 2021'].str.strip().str.lower() == 'clarence') &
    (merged_df['SA4 Name 2021'].str.strip().str.lower() == 'unknown'),
    'SA4 Name 2021'
] = 'Hobart'


C:\Users\Aparna\AppData\Local\Temp\ipykernel_21272\2641093565.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df[speed_col] = merged_df[speed_col].replace('<40', 40)


In [6]:

# Only convert strings to time, keep time objects as-is
merged_df['Time'] = merged_df['Time'].apply(
    lambda x: datetime.strptime(x, '%H:%M:%S').time() if isinstance(x, str) else x
)

# Define time-of-day logic
def get_time_of_day(t):
    if pd.isna(t):
        return 'Unknown'
    elif t.hour >= 6 and t.hour < 18:
        return 'Day'
    else:
        return 'Night'

# Apply the function
merged_df['Time of day'] = merged_df['Time'].apply(get_time_of_day)

# Create a Holiday Indicator column. 
def assign_holiday(row):
    if row['Christmas Period'] == 'Yes':
        return 'Christmas'
    elif row['Easter Period'] == 'Yes':
        return 'Easter'
    else:
        return 'Non-Holiday'

merged_df['Holiday Indicator'] = merged_df.apply(assign_holiday, axis=1)



In [7]:
# Fix Age Group for rows where Age = 19 and Age Group is Unknown
merged_df.loc[
    (merged_df['Age'] == 19) & (merged_df['Age Group'].str.lower() == 'unknown'),
    'Age Group'
] = '17_to_25'

# Fix Age where the Age Group is known (1_to_16)
missing_before = merged_df[
    (merged_df['Age Group'] == '1_to_16') & (merged_df['Age'].isna())
].shape[0]

print(f"Missing ages before imputation: {missing_before}")

# Make sure Age is numeric
merged_df['Age'] = pd.to_numeric(merged_df['Age'], errors='coerce')

# Filter for rows where Age Group is 1_to_16 and Age is NOT null
ages_in_group = merged_df.loc[
    (merged_df['Age Group'] == '1_to_16') & (merged_df['Age'].notna()),
    'Age'
]

# Calculate mode
age_mode_1_to_16 = ages_in_group.mode().iloc[0]  # Take first mode if multiple
print(f"Mode of Age in '1_to_16' group: {age_mode_1_to_16}")

# Fill missing Age values in this group with the mode
merged_df.loc[
    (merged_df['Age Group'] == '1_to_16') & (merged_df['Age'].isna()),
    'Age'
] = age_mode_1_to_16

missing_after = merged_df[
    (merged_df['Age Group'] == '1_to_16') & (merged_df['Age'].isna())
].shape[0]

print(f"Missing ages after imputation: {missing_after}")

rows_filled = missing_before - missing_after
print(f"Total rows filled with mode: {rows_filled}")


Missing ages before imputation: 232
Mode of Age in '1_to_16' group: 16.0
Missing ages after imputation: 0
Total rows filled with mode: 232


In [8]:
columns_to_check = [
    'Speed Limit',
    'National Remoteness Areas',
    'SA4 Name 2021',
    'National LGA Name 2021',
    'National Road Type'
]

# Define helper function to check "unknown" status
def is_unknown_or_blank(val):
    return pd.isna(val) or val == '' or str(val).strip().lower() == 'unknown' or str(val).strip().lower() == 'undetermined'

# Apply the function across all specified columns
filtered_rows = merged_df[
    merged_df[columns_to_check].applymap(is_unknown_or_blank).all(axis=1)
]

# Show how many such rows exist
print(f"Total rows where all 5 columns are unknown/blank/NaN: {len(filtered_rows)}")

# Original number of rows
total_rows = len(merged_df)

# Boolean mask for rows to remove
mask = merged_df[columns_to_check].applymap(is_unknown_or_blank).all(axis=1)

# Count rows to be removed
rows_to_remove = mask.sum()

# Calculate percentage removed
percent_removed = (rows_to_remove / total_rows) * 100

# Remove rows
merged_df_cleaned = merged_df[~mask]

# Print log
print(f"Total rows before cleaning: {total_rows}")
print(f"Rows removed: {rows_to_remove}")
print(f"Percentage removed: {percent_removed:.2f}%")
print(f"Remaining rows: {len(merged_df_cleaned)}")


C:\Users\Aparna\AppData\Local\Temp\ipykernel_21272\143186521.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  merged_df[columns_to_check].applymap(is_unknown_or_blank).all(axis=1)


Total rows where all 5 columns are unknown/blank/NaN: 1401


C:\Users\Aparna\AppData\Local\Temp\ipykernel_21272\143186521.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = merged_df[columns_to_check].applymap(is_unknown_or_blank).all(axis=1)


Total rows before cleaning: 56874
Rows removed: 1401
Percentage removed: 2.46%
Remaining rows: 55473


In [9]:
# Preview the merged DataFrame
print(merged_df_cleaned.head())

# Save to a new Excel
merged_df_cleaned.to_csv('merged_fatalities_crashes.csv', index=False)

   Crash ID State  Month  Year Dayweek      Time Crash Type Bus Involvement  \
0  20241115   NSW     12  2024  Friday  04:00:00     Single              No   
1  20241125   NSW     12  2024  Friday  06:15:00     Single              No   
2  20246013   TAS     12  2024  Friday  09:43:00   Multiple              No   
3  20241002   NSW     12  2024  Friday  10:35:00   Multiple              No   
5  20243185   QLD     12  2024  Friday  13:00:00   Multiple              No   

  Heavy Rigid Truck Involvement Articulated Truck Involvement  ...  \
0                            No                            No  ...   
1                            No                            No  ...   
2                            No                            No  ...   
3                            No                            No  ...   
5                            No                            No  ...   

  National LGA Name 2021         National Road Type Christmas Period  \
0            Wagga Wagga        

## Road User

In [10]:
# Step 1: Extract unique road user categories
dim_road_user = merged_df[['Road User']].drop_duplicates().reset_index(drop=True)

# Step 2: Generate road_user_id like RU1, RU2...
dim_road_user['road_user_id'] = ['RU' + str(i+1) for i in range(len(dim_road_user))]

# Step 3: Reorder columns
dim_road_user = dim_road_user[['road_user_id', 'Road User']]

dim_road_user.to_csv("dim_road_user.csv", index=False)


## National Road Type

In [11]:
# Step 1: Extract unique road types from the cleaned dataset
dim_national_road_type = merged_df_cleaned[['National Road Type']].drop_duplicates().reset_index(drop=True)

# Step 2: Create surrogate keys like NR1, NR2, ...
dim_national_road_type['national_road_type_id'] = ['NR' + str(i+1) for i in range(len(dim_national_road_type))]

# Step 3: Reorder columns
dim_national_road_type = dim_national_road_type[['national_road_type_id', 'National Road Type']]

dim_national_road_type.to_csv("dim_national_road_type.csv", index=False)

## Crash Type Dimesnion

In [12]:
# Step 1: Extract unique crash types from the cleaned dataset
dim_crash_type = merged_df_cleaned[['Crash Type']].drop_duplicates().reset_index(drop=True)

# Step 2: Create surrogate keys like CT1, CT2, ...
dim_crash_type['crash_type_id'] = ['CT' + str(i+1) for i in range(len(dim_crash_type))]

# Step 3: Reorder columns
dim_crash_type = dim_crash_type[['crash_type_id', 'Crash Type']]

dim_crash_type.to_csv("dim_crash_type.csv", index=False)

## Vehicle involvement Dimension

In [13]:

dim_vehicle = merged_df_cleaned[[
    'Bus Involvement',
    'Heavy Rigid Truck Involvement',
    'Articulated Truck Involvement',
    'Vehicle Involvement'
]].drop_duplicates().reset_index(drop=True)

# Add surrogate key
dim_vehicle['vehicle_involvement_id'] = ['VI' + str(i+1) for i in range(len(dim_vehicle))]

# Reorder columns
dim_vehicle = dim_vehicle[[
    'vehicle_involvement_id',
    'Bus Involvement',
    'Heavy Rigid Truck Involvement',
    'Articulated Truck Involvement',
    'Vehicle Involvement'
]]

# Save to CSV
dim_vehicle.to_csv("dim_vehicle_involvement.csv", index=False)


# Person Dimension

In [14]:
# Step 1: Select relevant columns from the cleaned fact table
dim_person = merged_df_cleaned[[
    'Gender',
    'Age',
    'Age Group'
]].drop_duplicates().reset_index(drop=True)

# Step 2: Create surrogate keys like P1, P2, ...
dim_person['person_id'] = ['P' + str(i+1) for i in range(len(dim_person))]

# Step 3: Reorder columns
dim_person = dim_person[[
    'person_id',
    'Gender',
    'Age',
    'Age Group'
]]

dim_person.to_csv("dim_person.csv", index=False)


## Date Dimension

In [15]:
# Step 1: Extract relevant columns
dim_date = merged_df_cleaned[[
    'Year', 'Month', 'Dayweek', 'Day of week', 'Holiday Indicator'
]].drop_duplicates().reset_index(drop=True)


# Step 3: Generate surrogate keys like D1, D2, D3...
dim_date['date_id'] = ['D' + str(i+1) for i in range(len(dim_date))]

# Step 4: Reorder columns
dim_date = dim_date[[
    'date_id', 'Year', 'Month', 'Dayweek', 'Day of week', 'Holiday Indicator'
]]

# Step 5: Save to CSV
dim_date.to_csv("dim_date.csv", index=False)


## LGA Dimension

In [16]:
dim_lga = pd.read_csv("LGA (count of dwellings).csv")

# Add surrogate key column
dim_lga['lga_id'] = ['LGA' + str(i + 1) for i in range(len(dim_lga))]

# Reorder columns to put lga_id first
dim_lga = dim_lga[['lga_id'] + [col for col in dim_lga.columns if col != 'lga_id']]

# Add Unknown row
unknown_row = pd.DataFrame({
    'lga_id': ['LGA0'],
    'LGA': ['Unknown'],
    'Dwellings Count': [0]
})

# Append and remove duplicates just in case
dim_lga = pd.concat([unknown_row, dim_lga], ignore_index=True).drop_duplicates()

# Save to CSV
dim_lga.to_csv("dim_lga.csv", index=False)


## Location Dimension

In [17]:
# Load dim_lga
dim_lga = pd.read_csv('dim_lga.csv')

# Step 1: Prepare dim_location base table
dim_location = merged_df_cleaned[[
    'National LGA Name 2021',
    'National Remoteness Areas',
    'SA4 Name 2021',
    'State'
]].drop_duplicates().reset_index(drop=True)

# Step 2: Strip 'Shire' and leading/trailing spaces for cleaner matching
dim_location['Cleaned LGA'] = dim_location['National LGA Name 2021'].str.replace('Shire', '', regex=False).str.strip()
dim_lga['Cleaned LGA'] = dim_lga['LGA'].str.replace('Shire', '', regex=False).str.strip()

# Step 3: Handle special mapping logic
def resolve_lga_name(row):
    name = row['Cleaned LGA']
    state = row['State']

    if name == 'Cootamundra-Gundagai':
        return 'Cootamundra-Gundagai Regional'
    elif name == 'Nambucca':
        return 'Nambucca Valley'
    elif name == 'Campbelltown':
        return 'Campbelltown (NSW)' if state == 'NSW' else 'Campbelltown (SA)'
    elif name == 'Unincorporated':
        return f'Unincorporated {state}'
    elif name == 'Bayside':
        return f'Bayside ({state})'
    elif name == 'Central Coast':
        return f'Central Coast ({state})'
    else:
        return name

dim_location['Resolved LGA Name'] = dim_location.apply(resolve_lga_name, axis=1)

# Step 4: Merge with dim_lga on Resolved Name
dim_location = dim_location.merge(
    dim_lga[['lga_id', 'LGA', 'Cleaned LGA']],
    left_on='Resolved LGA Name',
    right_on='Cleaned LGA',
    how='left'
)

# Step 5: Assign location_id
dim_location['location_id'] = ['L' + str(i + 1) for i in range(len(dim_location))]

# Step 6: Final cleanup
dim_location = dim_location[[
    'location_id', 'lga_id', 'State', 'National Remoteness Areas',
    'SA4 Name 2021', 'National LGA Name 2021'
]]

# Save to CSV
dim_location.to_csv('dim_location.csv', index=False)



In [18]:
dim_national_road_type.head()

,national_road_type_id,National Road Type
0,NR1,Arterial Road
1,NR2,Local Road
2,NR3,National or State Highway
3,NR4,Sub-arterial Road
4,NR5,Unknown


In [19]:
dim_location.head()

,location_id,lga_id,State,National Remoteness Areas,SA4 Name 2021,National LGA Name 2021
0,L1,LGA115,NSW,Inner Regional Australia,Riverina,Wagga Wagga
1,L2,LGA52,NSW,Inner Regional Australia,Sydney - Baulkham Hills and Hawkesbury,Hawkesbury
2,L3,LGA525,TAS,Inner Regional Australia,Launceston and North East,Northern Midlands
3,L4,LGA2,NSW,Outer Regional Australia,New England and North West,Armidale Regional
4,L5,LGA250,QLD,Inner Regional Australia,Toowoomba,Lockyer Valley


In [20]:
dim_lga.head()

,lga_id,LGA,Dwellings Count,Cleaned LGA
0,LGA0,Unknown,0,Unknown
1,LGA1,Albury,25430,Albury
2,LGA2,Armidale Regional,12955,Armidale Regional
3,LGA3,Ballina,20889,Ballina
4,LGA4,Balranald,1091,Balranald


In [21]:
dim_date.head()

,date_id,Year,Month,Dayweek,Day of week,Holiday Indicator
0,D1,2024,12,Friday,Weekday,Christmas
1,D2,2024,12,Friday,Weekday,Non-Holiday
2,D3,2024,12,Monday,Weekday,Non-Holiday
3,D4,2024,12,Monday,Weekday,Christmas
4,D5,2024,12,Saturday,Weekend,Non-Holiday


In [22]:
dim_vehicle.head()

,vehicle_involvement_id,Bus Involvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,Vehicle Involvement
0,VI1,No,No,No,No Heavy Vehicles Involved
1,VI2,No,No,Yes,Only Articulated Truck Involved as Known
2,VI3,Unknown,Unknown,Unknown,Unknown
3,VI4,Yes,No,Yes,Bus and Articulated Truck Involved as Known
4,VI5,No,Yes,No,Only Heavy Rigid Truck Involved as Known


In [23]:
dim_person.head()

,person_id,Gender,Age,Age Group
0,P1,Male,74.0,65_to_74
1,P2,Female,19.0,17_to_25
2,P3,Female,33.0,26_to_39
3,P4,Female,32.0,26_to_39
4,P5,Female,61.0,40_to_64


In [24]:
dim_road_user.head()

,road_user_id,Road User
0,RU1,Driver
1,RU2,Passenger
2,RU3,Motorcycle rider
3,RU4,Pedestrian
4,RU5,Pedal cyclist


In [25]:
dim_crash_type.head()

,crash_type_id,Crash Type
0,CT1,Single
1,CT2,Multiple


In [26]:
# Load main data
merged_df_cleaned = pd.read_csv("merged_fatalities_crashes.csv")

# Load dimension tables
dim_date = pd.read_csv("dim_date.csv")
dim_location = pd.read_csv("dim_location.csv")
dim_crash_type = pd.read_csv("dim_crash_type.csv")
dim_person = pd.read_csv("dim_person.csv")
dim_road_user = pd.read_csv("dim_road_user.csv")
dim_vehicle = pd.read_csv("dim_vehicle_involvement.csv")
dim_road_type = pd.read_csv("dim_national_road_type.csv")

# Make a copy to start joining
fact_df = merged_df_cleaned.copy()

# --- Join with dimension tables to get IDs ---
fact_df = fact_df.merge(
    dim_date,
    on=['Year', 'Month', 'Dayweek', 'Day of week', 'Holiday Indicator'],
    how='left'
)

fact_df = fact_df.merge(
    dim_location,
    on=['State', 'National Remoteness Areas', 'SA4 Name 2021', 'National LGA Name 2021'],
    how='left'
)

fact_df = fact_df.merge(
    dim_crash_type,
    on='Crash Type',
    how='left'
)

fact_df = fact_df.merge(
    dim_person,
    on=['Gender', 'Age', 'Age Group'],
    how='left'
)

fact_df = fact_df.merge(
    dim_road_user,
    left_on='Road User',
    right_on='Road User',
    how='left'
)

fact_df = fact_df.merge(
    dim_vehicle,
    on=['Bus Involvement', 'Heavy Rigid Truck Involvement', 'Articulated Truck Involvement', 'Vehicle Involvement'],
    how='left'
)

fact_df = fact_df.merge(
    dim_road_type,
    on='National Road Type',
    how='left'
)

# Select relevant columns for the fact table
fact_crash_df = fact_df[[
    'Crash ID',
    'date_id',
    'location_id',
    'road_user_id',
    'person_id',
    'crash_type_id',
    'vehicle_involvement_id',
    'national_road_type_id',
    'Time',              # new
    'Time of day',       # new
    'Speed Limit',       # new
    'Number Fatalities'
]]

# Rename columns to match DB schema
fact_crash_df = fact_crash_df.rename(columns={
    'Crash ID': 'crash_id',
    'Time': 'time',
    'Time of day': 'time_of_day',
    'Speed Limit': 'speed_limit',
    'Number Fatalities': 'number_fatalities'
})

# Save to CSV (no index)
fact_crash_df.to_csv('fact_crash_fatalities.csv', index=False)



In [27]:
fact_crash_df.head()

,crash_id,date_id,location_id,road_user_id,person_id,crash_type_id,vehicle_involvement_id,national_road_type_id,time,time_of_day,speed_limit,number_fatalities
0,20241115,D1,L1,RU1,P1,CT1,VI1,NR1,04:00:00,Night,100.0,1
1,20241125,D2,L2,RU1,P2,CT1,VI1,NR2,06:15:00,Day,80.0,1
2,20246013,D1,L3,RU1,P3,CT2,VI1,NR2,09:43:00,Day,50.0,1
3,20241002,D2,L4,RU1,P4,CT2,VI1,NR3,10:35:00,Day,100.0,1
4,20243185,D2,L5,RU2,P5,CT2,VI1,NR3,13:00:00,Day,100.0,1


In [41]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Load your dataset
df = pd.read_csv("merged_fatalities_crashes.csv")

# Select relevant categorical columns (keep it compact and relevant)
cols = ["Gender", "Age Group", "Time of day", "Day of week", "Crash Type", 
    "Vehicle Involvement", "Holiday Indicator", "Road User"]

# Drop rows with unknowns (optional: depends on your dataset's distribution)
df = df[df['Road User'] != 'Unknown']

# Convert to list of transactions
transactions = df[cols].astype(str).values.tolist()

# Transaction encoding
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# Apply Apriori (very low threshold just to capture rules)
frequent_itemsets = apriori(df_encoded, min_support=0.005, use_colnames=True)

# Generate rules with very loose thresholds
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)


rules_road_user = rules[rules['consequents'].apply(lambda x: any(item in ['Driver', 'Passenger', 'Pedestrian', 'Motorcycle rider', 'Pedal cyclist'] for item in x))]


# Filter only rules with 'Road User=...' in the CONSEQUENT
# rules = rules[rules['consequents'].astype(str).str.contains("Road User")]

# Sort by lift and confidence
rules = rules.sort_values(by=["lift", "confidence"], ascending=False)

# Display top rules
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(5))


                                          antecedents  \
8192                        (Male, Multiple, 1_to_16)   
8199                                  (Pedal cyclist)   
107716  (Day, Pedestrian, No Heavy Vehicles Involved)   
107704                  (75_or_older, Single, Female)   
142415  (Day, Pedestrian, No Heavy Vehicles Involved)   

                                          consequents   support  confidence  \
8192                                  (Pedal cyclist)  0.005012    0.265521   
8199                        (Male, Multiple, 1_to_16)  0.005012    0.183377   
107716                  (75_or_older, Single, Female)  0.006022    0.159580   
107704  (Day, Pedestrian, No Heavy Vehicles Involved)  0.006022    0.300090   
142415     (Non-Holiday, 75_or_older, Single, Female)  0.005788    0.153368   

            lift  
8192    9.713918  
8199    9.713918  
107716  7.952023  
107704  7.952023  
142415  7.905313  


In [43]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Load your cleaned dataset
df = pd.read_csv("merged_fatalities_crashes.csv")

# Filter to only relevant categorical columns
cols = ["Gender", "Age Group", "Time of day", "Day of week", "Crash Type", 
        "Vehicle Involvement", "Holiday Indicator", "Road User"]

df_subset = df[cols].astype(str)

# Convert each row into a list of attribute=value for better readability
transactions = df_subset.apply(lambda row: [f"{col}={val}" for col, val in row.items()], axis=1).tolist()

# One-hot encode using TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# Frequent Itemsets with low threshold for exploration
frequent_itemsets = apriori(df_encoded, min_support=0.005, use_colnames=True)

# Generate rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Keep only rules where RHS has a Road User
rules = rules[rules['consequents'].apply(lambda x: any("Road User=" in item for item in x))]

# Sort by confidence and lift
rules_sorted = rules.sort_values(by=["lift", "confidence"], ascending=False)

# Make output prettier
def format_set(s):
    return ', '.join(list(s))

rules_sorted["antecedents"] = rules_sorted["antecedents"].apply(format_set)
rules_sorted["consequents"] = rules_sorted["consequents"].apply(format_set)

# Show the top k rules (you can change k)
k = 5
pretty_rules = rules_sorted[["antecedents", "consequents", "support", "confidence", "lift"]].head(k)

# Display the result
print(pretty_rules.to_string(index=False))


                                                                           antecedents                                                                                                          consequents  support  confidence     lift
                                   Gender=Male, Age Group=1_to_16, Crash Type=Multiple                                                                                              Road User=Pedal cyclist 0.005011    0.265267 9.706574
                               Age Group=75_or_older, Gender=Female, Crash Type=Single                                Vehicle Involvement=No Heavy Vehicles Involved, Road User=Pedestrian, Time of day=Day 0.006021    0.300090 7.953600
Crash Type=Single, Age Group=75_or_older, Gender=Female, Holiday Indicator=Non-Holiday                                Vehicle Involvement=No Heavy Vehicles Involved, Road User=Pedestrian, Time of day=Day 0.005787    0.298327 7.906881
                               Age Group=75_or_older, Gender=Fem

In [44]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Load data
df = pd.read_csv("merged_fatalities_crashes.csv")

# Select relevant categorical columns
cols = ["Gender", "Age Group", "Time of day", "Day of week", "Crash Type", 
        "Vehicle Involvement", "Holiday Indicator", "Road User"]
df_subset = df[cols].astype(str)

# Convert rows into list of formatted strings like "Gender=Male"
transactions = df_subset.apply(lambda row: [f"{col}={val}" for col, val in row.items()], axis=1).tolist()

# One-hot encode
te = TransactionEncoder()
te_ary = te.fit_transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# Run Apriori
frequent_itemsets = apriori(df_encoded, min_support=0.1, use_colnames=True)

# Generate rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Filter for Road User in RHS
rules = rules[rules['consequents'].apply(lambda x: any("Road User=" in item for item in x))]

# Sort rules by lift and confidence
rules_sorted = rules.sort_values(by=["lift", "confidence"], ascending=False)

# Format for pretty output
rules_sorted["antecedents"] = rules_sorted["antecedents"].apply(lambda x: ', '.join(sorted(list(x))))
rules_sorted["consequents"] = rules_sorted["consequents"].apply(lambda x: ', '.join(sorted(list(x))))
rules_sorted[["support", "confidence", "lift"]] = rules_sorted[["support", "confidence", "lift"]].round(3)

# Select final columns
pretty_rules = rules_sorted[["antecedents", "consequents", "support", "confidence", "lift"]].head(10)

# Optional: export to CSV or styled HTML
pretty_rules.to_csv("top_road_user_rules.csv", index=False)

# Display nicely in notebook
from IPython.display import display
display(pretty_rules.style.set_caption("Top Association Rules with Road User as Consequent"))


,antecedents,consequents,support,confidence,lift
75,Gender=Female,Road User=Passenger,0.107000,0.379000,1.685000
527,Gender=Female,"Holiday Indicator=Non-Holiday, Road User=Passenger",0.102000,0.361000,1.681000
525,"Gender=Female, Holiday Indicator=Non-Holiday",Road User=Passenger,0.102000,0.376000,1.674000
385,"Crash Type=Single, Holiday Indicator=Non-Holiday",Road User=Pedestrian,0.130000,0.245000,1.593000
49,Crash Type=Single,Road User=Pedestrian,0.134000,0.243000,1.581000
389,Crash Type=Single,"Holiday Indicator=Non-Holiday, Road User=Pedestrian",0.130000,0.236000,1.580000
1385,"Crash Type=Multiple, Day of week=Weekday","Holiday Indicator=Non-Holiday, Road User=Driver, Time of day=Day",0.111000,0.352000,1.410000
1371,"Crash Type=Multiple, Day of week=Weekday, Holiday Indicator=Non-Holiday","Road User=Driver, Time of day=Day",0.111000,0.365000,1.407000
784,"Crash Type=Multiple, Day of week=Weekday","Road User=Driver, Time of day=Day",0.115000,0.364000,1.405000
1461,"Gender=Male, Time of day=Night","Crash Type=Single, Holiday Indicator=Non-Holiday, Road User=Driver",0.105000,0.314000,1.375000
